# **FLUX UPSCALER - 2x EVALUATION MODE (Local - RTX 5060 Ti)**

## Purpose
This notebook upscales images to **2x the target print size** for quality evaluation in Photoshop.

- 11x14" target → outputs at 22x28" for inspection
- 10x20" target → outputs at 20x40" for inspection

This allows you to zoom to 100% in Photoshop and check for pixelation before committing to the final upscale.

## Setup Instructions
1. Run the **Setup Environment** cell first (only needed once)
2. Set your `IMAGE_CONFIGS` with the **original target sizes** (not 2x)
3. The notebook automatically calculates 2x dimensions
4. Run the **Upscale** cell

## Notes
- Models are downloaded to `./hf/` directory (relative to this repo)
- Output is saved with `_2x-eval` suffix for easy identification
- Use the standard notebook for final production output

In [1]:
# @title Setup Environment (Run Once)
import subprocess
import sys
import os
from pathlib import Path

# Get repo root (parent of resolution-upscaling folder)
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

# Directory paths
HF_DIR = REPO_ROOT / "hf"
COMFYUI_DIR = HF_DIR / "ComfyUI"
MODELS_DIR = HF_DIR / "models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Create directories
for d in [HF_DIR, MODELS_DIR, OUTPUT_DIR, INPUT_DIR, 
          MODELS_DIR / "upscale_models", MODELS_DIR / "unet", 
          MODELS_DIR / "vae", MODELS_DIR / "clip", MODELS_DIR / "loras"]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Repo root: {REPO_ROOT}")
print(f"Models directory: {MODELS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

def install_pip_packages():
    # Install PyTorch NIGHTLY with CUDA 12.8 (required for RTX 5060 Ti sm_120 Blackwell)
    print("Installing PyTorch nightly with CUDA 12.8 (for RTX 50-series)...")
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', '-q', '--pre',
         'torch', 'torchvision', 'torchaudio',
         '--index-url', 'https://download.pytorch.org/whl/nightly/cu128'],
        capture_output=True
    )
    if result.returncode == 0:
        print("✓ PyTorch nightly with CUDA 12.8 installed")
    else:
        print(f"✗ PyTorch install failed: {result.stderr.decode()}")
        raise RuntimeError("PyTorch CUDA installation failed")
    
    # Other packages (xformers removed - optional and causes conflicts)
    packages = [
        'torchsde',
        'av',
        'diffusers',
        'accelerate',
        'einops',
        'spandrel',
        'opencv-python',
        'imageio',
        'imageio-ffmpeg',
        'huggingface_hub',
        'safetensors',
        'gguf',
        'sentencepiece',
    ]

    for package in packages:
        try:
            subprocess.run(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                check=True,
                capture_output=True
            )
            print(f"✓ {package} installed")
        except subprocess.CalledProcessError as e:
            print(f"✗ Error installing {package}: {e.stderr.decode().strip() or 'Unknown error'}")

print("Installing pip packages...")
install_pip_packages()

# Clone ComfyUI and custom nodes if not present
if not COMFYUI_DIR.exists():
    print("Cloning ComfyUI...")
    subprocess.run(['git', 'clone', 'https://github.com/Isi-dev/ComfyUI', str(COMFYUI_DIR)], check=True)
    
custom_nodes_dir = COMFYUI_DIR / "custom_nodes"
custom_nodes_dir.mkdir(exist_ok=True)

if not (custom_nodes_dir / "ComfyUI_GGUF").exists():
    print("Cloning ComfyUI_GGUF...")
    subprocess.run(['git', 'clone', 'https://github.com/Isi-dev/ComfyUI_GGUF.git'], cwd=str(custom_nodes_dir), check=True)
    # Install GGUF requirements
    req_file = custom_nodes_dir / "ComfyUI_GGUF" / "requirements.txt"
    if req_file.exists():
        subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-r', str(req_file)], check=True)

if not (custom_nodes_dir / "ComfyUI_UltimateSDUpscale").exists():
    print("Cloning ComfyUI_UltimateSDUpscale...")
    subprocess.run(['git', 'clone', 'https://github.com/Isi-dev/ComfyUI_UltimateSDUpscale'], cwd=str(custom_nodes_dir), check=True)

print("\n✅ Environment Setup Complete!")

Repo root: C:\Users\Armaan\Desktop\Artinafti
Models directory: C:\Users\Armaan\Desktop\Artinafti\hf\models
Output directory: C:\Users\Armaan\Desktop\Artinafti\4xoutput
Installing pip packages...
Installing PyTorch nightly with CUDA 12.8 (for RTX 50-series)...
✓ PyTorch nightly with CUDA 12.8 installed
✓ torchsde installed
✓ av installed
✓ diffusers installed
✓ accelerate installed
✓ einops installed
✓ spandrel installed
✓ opencv-python installed
✓ imageio installed
✓ imageio-ffmpeg installed
✓ huggingface_hub installed
✓ safetensors installed
✓ gguf installed
✓ sentencepiece installed

✅ Environment Setup Complete!


In [2]:
# @title Download Models (Run Once)
import os
from pathlib import Path
from huggingface_hub import hf_hub_download

# Get paths from previous cell
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

HF_DIR = REPO_ROOT / "hf"
MODELS_DIR = HF_DIR / "models"

# Create model subdirectories
UPSCALE_MODELS_DIR = MODELS_DIR / "upscale_models"
UNET_DIR = MODELS_DIR / "unet"
VAE_DIR = MODELS_DIR / "vae"
CLIP_DIR = MODELS_DIR / "clip"
LORAS_DIR = MODELS_DIR / "loras"

for d in [UPSCALE_MODELS_DIR, UNET_DIR, VAE_DIR, CLIP_DIR, LORAS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Configuration
download_face_upscalers = False  # Set to True if you want face upscalers
download_loRA = False  # Set to True if you want LoRA

def download_model(repo_id: str, filename: str, dest_dir: Path, subfolder: str = None) -> str:
    """Download model from HuggingFace Hub."""
    dest_path = dest_dir / filename
    if dest_path.exists():
        print(f"✓ {filename} already exists")
        return filename
    
    try:
        print(f"Downloading {filename}...", end=' ', flush=True)
        hf_hub_download(
            repo_id=repo_id,
            filename=filename if subfolder is None else f"{subfolder}/{filename}",
            local_dir=str(dest_dir),
            local_dir_use_symlinks=False
        )
        # Move file if it was downloaded to subfolder
        if subfolder:
            src = dest_dir / subfolder / filename
            if src.exists():
                src.rename(dest_path)
                (dest_dir / subfolder).rmdir()
        print("Done!")
        return filename
    except Exception as e:
        print(f"\nError downloading {filename}: {e}")
        return None

print("Downloading upscale models...")
x_UltraSharp = download_model("Isi99999/Upscalers", "4x-UltraSharp.pth", UPSCALE_MODELS_DIR)
x_foolhardy_Remacri = download_model("Isi99999/Upscalers", "4x_foolhardy_Remacri.pth", UPSCALE_MODELS_DIR)
x_AnimeSharp = download_model("Isi99999/Upscalers", "4x-AnimeSharp.pth", UPSCALE_MODELS_DIR)

if download_face_upscalers:
    x_FaceUpSharpDAT = download_model("Isi99999/Upscalers", "4xFaceUpSharpDAT.pth", UPSCALE_MODELS_DIR)
    x_FaceUpSharpLDAT = download_model("Isi99999/Upscalers", "4xFaceUpSharpLDAT.safetensors", UPSCALE_MODELS_DIR)

print("\nDownloading FLUX model...")
flux_model = download_model("city96/FLUX.1-dev-gguf", "flux1-dev-Q8_0.gguf", UNET_DIR)

print("\nDownloading VAE...")
flux_vae = download_model("Isi99999/Upscalers", "ae.sft", VAE_DIR, subfolder="Flux")

print("\nDownloading CLIP models...")
flux_clip_l = download_model("Isi99999/Upscalers", "clip_l.safetensors", CLIP_DIR, subfolder="Flux")
flux_t5xxl = download_model("Isi99999/Upscalers", "t5xxl_fp8_e4m3fn.safetensors", CLIP_DIR, subfolder="Flux")

if download_loRA:
    print("\nDownloading LoRA...")
    flux_lora = download_model("Isi99999/Upscalers", "flux_realism_lora.safetensors", LORAS_DIR, subfolder="Flux")

print("\n✅ All models downloaded!")

✓ 4x-UltraSharp.pth already exists
✓ 4x_foolhardy_Remacri.pth already exists
✓ 4x-AnimeSharp.pth already exists

✓ flux1-dev-Q8_0.gguf already exists

✓ ae.sft already exists

✓ clip_l.safetensors already exists
✓ t5xxl_fp8_e4m3fn.safetensors already exists

✅ All models downloaded!


In [ ]:
# @title Load Libraries and Initialize
import torch
import numpy as np
from PIL import Image
import gc
import os
import sys
import random
import imageio
import cv2
import shutil
import datetime
from pathlib import Path
from IPython.display import display, HTML, Image as IPImage

# Set up paths
NOTEBOOK_DIR = Path(os.getcwd()).resolve()
if NOTEBOOK_DIR.name == "resolution-upscaling":
    REPO_ROOT = NOTEBOOK_DIR.parent
else:
    REPO_ROOT = NOTEBOOK_DIR

HF_DIR = REPO_ROOT / "hf"
COMFYUI_DIR = HF_DIR / "ComfyUI"
MODELS_DIR = HF_DIR / "models"
OUTPUT_DIR = REPO_ROOT / "4xoutput"
INPUT_DIR = REPO_ROOT / "input"

# Model paths
UPSCALE_MODELS_DIR = MODELS_DIR / "upscale_models"
UNET_DIR = MODELS_DIR / "unet"
VAE_DIR = MODELS_DIR / "vae"
CLIP_DIR = MODELS_DIR / "clip"
LORAS_DIR = MODELS_DIR / "loras"

# Add ComfyUI to path
sys.path.insert(0, str(COMFYUI_DIR))

# Configure ComfyUI folder_paths to use our custom model directories
import folder_paths
folder_paths.folder_names_and_paths["text_encoders"] = ([str(CLIP_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["clip"] = ([str(CLIP_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["vae"] = ([str(VAE_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["diffusion_models"] = ([str(UNET_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["unet"] = ([str(UNET_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["upscale_models"] = ([str(UPSCALE_MODELS_DIR)], folder_paths.supported_pt_extensions)
folder_paths.folder_names_and_paths["loras"] = ([str(LORAS_DIR)], folder_paths.supported_pt_extensions)

# Import ComfyUI modules
from nodes import (
    DualCLIPLoader,
    UNETLoader,
    VAELoader,
    LoraLoaderModelOnly,
    LoadImage,
    SaveImage
)

from custom_nodes.ComfyUI_GGUF.nodes import UnetLoaderGGUF
from comfy_extras.nodes_upscale_model import UpscaleModelLoader
from comfy_extras.nodes_flux import CLIPTextEncodeFlux
from custom_nodes.ComfyUI_UltimateSDUpscale.nodes import (
    UltimateSDUpscale,
    UltimateSDUpscaleNoUpscale
)

# Model filenames
flux_model = "flux1-dev-Q8_0.gguf"
flux_vae = "ae.sft"
flux_clip_l = "clip_l.safetensors"
flux_t5xxl = "t5xxl_fp8_e4m3fn.safetensors"
lora = None  # Set if using LoRA

# Initialize loaders
clip_loader = DualCLIPLoader()
unet_loader = UnetLoaderGGUF()
vae_loader = VAELoader()
load_lora = LoraLoaderModelOnly()
load_image = LoadImage()
save_image = SaveImage()
upscale_model_loader = UpscaleModelLoader()
positive_prompt_encode = CLIPTextEncodeFlux()
negative_prompt_encode = CLIPTextEncodeFlux()
upscaler = UltimateSDUpscale()
noUpscale = UltimateSDUpscaleNoUpscale()

# Helper functions
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
    gc.collect()

def save_as_image(image, filename_prefix, output_dir=None, formats=None):
    """Save single frame as PNG and/or TIFF image.

    Args:
        image: Image tensor to save
        filename_prefix: Base filename without extension
        output_dir: Output directory (defaults to OUTPUT_DIR)
        formats: List of formats to save, e.g. ["png", "tiff"] (defaults to ["png", "tiff"])

    Returns:
        List of saved file paths
    """
    if formats is None:
        formats = ["png", "tiff"]
    if output_dir is None:
        output_dir = OUTPUT_DIR
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    frame = (image.cpu().numpy() * 255).astype(np.uint8)
    pil_img = Image.fromarray(frame)

    saved_paths = []
    for fmt in formats:
        fmt_lower = fmt.lower()
        if fmt_lower == "png":
            output_path = output_dir / f"{filename_prefix}.png"
            pil_img.save(str(output_path), "PNG")
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")
        elif fmt_lower in ["tiff", "tif"]:
            output_path = output_dir / f"{filename_prefix}.tiff"
            pil_img.save(str(output_path), "TIFF", compression=None)
            saved_paths.append(str(output_path))
            print(f"   Saved: {output_path.name}")

    return saved_paths

def save_as_mp4(images, filename_prefix, fps, output_dir=None):
    """Save frames as MP4 video."""
    if output_dir is None:
        output_dir = OUTPUT_DIR
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / f"{filename_prefix}.mp4"

    frames = []
    for i, img in enumerate(images):
        try:
            if isinstance(img, torch.Tensor):
                img = img.cpu().numpy()

            if img.max() <= 1.0:
                img = (img * 255).astype(np.uint8)
            else:
                img = img.astype(np.uint8)

            if len(img.shape) == 4:
                img = img[0]

            if len(img.shape) == 3:
                if img.shape[0] in (1, 3, 4):
                    img = np.transpose(img, (1, 2, 0))
                elif img.shape[2] > 4:
                    img = img[:, :, :3]
            elif len(img.shape) == 2:
                img = np.expand_dims(img, axis=-1)

            if len(img.shape) != 3 or img.shape[2] not in (1, 3, 4):
                raise ValueError(f"Invalid frame shape after processing: {img.shape}")

            frames.append(img)
        except Exception as e:
            print(f"Error processing frame {i}: {str(e)}")
            raise

    with imageio.get_writer(str(output_path), fps=fps) as writer:
        for frame in frames:
            writer.append_data(frame)

    return str(output_path)

def extract_frames(video_path, max_frames=None):
    """Extract frames from video and return as a list of images."""
    vidcap = cv2.VideoCapture(str(video_path))
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frames = []

    while True:
        success, frame = vidcap.read()
        if not success or (max_frames and len(frames) >= max_frames):
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    vidcap.release()
    if not frames:
        return None, fps

    print(f"Extracted {len(frames)} frames")
    return frames, fps

def select_every_n_frame(frames, fps, n, skip_first=0, max_output_frames=0):
    """Select every nth frame from video."""
    if not frames or n < 1:
        raise ValueError("Frames must be a non-empty list and n must be >= 1")

    frames_to_use = frames[skip_first:]
    if not frames_to_use:
        print("No frames available after skipping.")
        return [], 0.0

    selected_frames = frames_to_use[::n]
    if max_output_frames != 0 and len(selected_frames) > max_output_frames:
        selected_frames = selected_frames[:max_output_frames]

    adjusted_fps = fps / n
    print(f"Adjusted FPS: {adjusted_fps:.2f} -> Final output: {len(selected_frames)} frames")

    return selected_frames, adjusted_fps

def display_video(video_path):
    """Display video in notebook."""
    from base64 import b64encode

    video_data = open(video_path, 'rb').read()
    if video_path.lower().endswith('.mp4'):
        mime_type = "video/mp4"
    elif video_path.lower().endswith('.webm'):
        mime_type = "video/webm"
    else:
        mime_type = "video/mp4"

    data_url = f"data:{mime_type};base64," + b64encode(video_data).decode()
    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="{mime_type}">
    </video>
    """))

def remove_frame(image_path: str, threshold: float = 0.15, min_frame_width: int = 10, max_frame_width: int = 200) -> str:
    """
    Remove frame/border from an image by detecting and cropping out the frame.
    
    Uses edge detection and variance analysis to find where the frame ends and content begins.
    
    Args:
        image_path: Path to input image
        threshold: Edge detection threshold (0.0-1.0), lower = more sensitive
        min_frame_width: Minimum expected frame width in pixels
        max_frame_width: Maximum expected frame width in pixels
    
    Returns:
        Path to the cropped image (saves to same location with '_no_frame' suffix)
    """
    image_path = Path(image_path)
    if not image_path.exists():
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    # Load image
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    original_height, original_width = img.shape[:2]
    
    # Convert to grayscale for analysis
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    def find_frame_edge(scan_range: int, is_horizontal: bool, from_start: bool) -> int:
        """
        Find frame edge by scanning from edge inward.
        
        Args:
            scan_range: How many pixels to scan from the edge
            is_horizontal: True for top/bottom edges, False for left/right
            from_start: True for top/left, False for bottom/right
        
        Returns:
            Pixel position where content starts (frame ends)
        """
        strip_width = 10
        max_variance_pos = 0
        max_variance = 0
        
        if is_horizontal:
            if from_start:
                # Scan from top
                for y in range(0, min(scan_range, original_height - strip_width), 2):
                    strip = gray[y:y+strip_width, :]
                    variance = np.var(strip)
                    if variance > max_variance:
                        max_variance = variance
                        max_variance_pos = y
            else:
                # Scan from bottom
                for y in range(original_height - min(scan_range, original_height), original_height - strip_width, -2):
                    strip = gray[y:y+strip_width, :]
                    variance = np.var(strip)
                    if variance > max_variance:
                        max_variance = variance
                        max_variance_pos = y
        else:
            if from_start:
                # Scan from left
                for x in range(0, min(scan_range, original_width - strip_width), 2):
                    strip = gray[:, x:x+strip_width]
                    variance = np.var(strip)
                    if variance > max_variance:
                        max_variance = variance
                        max_variance_pos = x
            else:
                # Scan from right
                for x in range(original_width - min(scan_range, original_width), original_width - strip_width, -2):
                    strip = gray[:, x:x+strip_width]
                    variance = np.var(strip)
                    if variance > max_variance:
                        max_variance = variance
                        max_variance_pos = x
        
        return max_variance_pos
    
    # Find frame edges
    scan_range = min(max_frame_width, original_width // 3, original_height // 3)
    
    top = find_frame_edge(scan_range, True, True)
    bottom = find_frame_edge(scan_range, True, False)
    left = find_frame_edge(scan_range, False, True)
    right = find_frame_edge(scan_range, False, False)
    
    # Ensure minimum frame width
    if top < min_frame_width:
        top = min_frame_width
    if left < min_frame_width:
        left = min_frame_width
    if original_height - bottom < min_frame_width:
        bottom = original_height - min_frame_width
    if original_width - right < min_frame_width:
        right = original_width - min_frame_width
    
    # Ensure valid bounds
    top = max(0, min(top, original_height - 20))
    bottom = max(top + 20, min(bottom, original_height))
    left = max(0, min(left, original_width - 20))
    right = max(left + 20, min(right, original_width))
    
    # Validate crop size (must be at least 50% of original)
    content_width = right - left
    content_height = bottom - top
    
    if content_width < original_width * 0.5 or content_height < original_height * 0.5:
        # Fallback: conservative crop (remove 3% from each side)
        margin = min(original_width, original_height) // 30
        top = margin
        bottom = original_height - margin
        left = margin
        right = original_width - margin
        print("Warning: Frame detection may have failed, using conservative crop")
    
    # Crop the image
    cropped = img[top:bottom, left:right]
    
    # Save cropped image
    output_path = image_path.parent / f"{image_path.stem}_no_frame{image_path.suffix}"
    cv2.imwrite(str(output_path), cropped)
    
    print(f"Removed frame: cropped from {original_width}x{original_height} to {content_width}x{content_height}")
    print(f"Removed margins: top={top}, bottom={original_height-bottom}, left={left}, right={original_width-right}")
    
    return str(output_path)

print(f"✅ Libraries loaded!")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# @title Configuration - 2x EVALUATION MODE

# ============== 2x EVALUATION MODE ==============
# This notebook upscales to 2x the target print size for quality inspection.
# Enter your ORIGINAL target sizes below - the notebook automatically doubles them.
#
# Examples:
#   11x14" target → evaluates at 22x28"
#   10x20" target → evaluates at 20x40"

EVALUATION_MULTIPLIER = 2  # 2x target dimensions for quality testing

DPI = 150

IMAGE_CONFIGS = [
    {
        "image_id": "b46bc519-9b4b-487f-b8d4-b95195d7e02e",
        "input_path": "input/b46bc519-9b4b-487f-b8d4-b95195d7e02e.jpg",
        "width_inches": 20,   # ORIGINAL target (will evaluate at 40")
        "height_inches": 10,  # ORIGINAL target (will evaluate at 20")
    },
]

# Calculate 2x evaluation dimensions
for config in IMAGE_CONFIGS:
    config["original_width"] = config["width_inches"]
    config["original_height"] = config["height_inches"]
    config["eval_width_inches"] = config["width_inches"] * EVALUATION_MULTIPLIER
    config["eval_height_inches"] = config["height_inches"] * EVALUATION_MULTIPLIER
    config["target_width_px"] = config["eval_width_inches"] * DPI
    config["target_height_px"] = config["eval_height_inches"] * DPI
    config["target_aspect"] = config["width_inches"] / config["height_inches"]


def calculate_scale_for_crop(
    input_width: int,
    input_height: int,
    target_width_inches: float,
    target_height_inches: float,
    dpi: int
) -> dict:
    """
    Calculate the scale factor and output dimensions needed to ensure
    we have enough resolution after cropping to the target aspect ratio.
    """
    # Target final dimensions
    final_width_px = int(target_width_inches * dpi)
    final_height_px = int(target_height_inches * dpi)
    
    # Aspect ratios (width/height)
    input_aspect = input_width / input_height
    target_aspect = target_width_inches / target_height_inches
    
    if abs(input_aspect - target_aspect) < 0.01:
        scale_factor = final_width_px / input_width
        return {
            "output_width_px": final_width_px,
            "output_height_px": final_height_px,
            "final_width_px": final_width_px,
            "final_height_px": final_height_px,
            "scale_factor": scale_factor,
            "crop_direction": "none",
            "crop_amount_px": 0,
            "crop_amount_inches": 0.0,
            "equivalent_print_size": f"{target_width_inches}\" x {target_height_inches}\""
        }
    
    if input_aspect < target_aspect:
        output_width_px = final_width_px
        scale_factor = output_width_px / input_width
        output_height_px = int(input_height * scale_factor)
        
        crop_amount_px = output_height_px - final_height_px
        crop_amount_inches = crop_amount_px / dpi
        crop_direction = "vertical"
        equivalent_height_inches = output_height_px / dpi
        equivalent_print_size = f"{target_width_inches}\" x {equivalent_height_inches:.2f}\""
    else:
        output_height_px = final_height_px
        scale_factor = output_height_px / input_height
        output_width_px = int(input_width * scale_factor)
        
        crop_amount_px = output_width_px - final_width_px
        crop_amount_inches = crop_amount_px / dpi
        crop_direction = "horizontal"
        equivalent_width_inches = output_width_px / dpi
        equivalent_print_size = f"{equivalent_width_inches:.2f}\" x {target_height_inches}\""
    
    return {
        "output_width_px": output_width_px,
        "output_height_px": output_height_px,
        "final_width_px": final_width_px,
        "final_height_px": final_height_px,
        "scale_factor": scale_factor,
        "crop_direction": crop_direction,
        "crop_amount_px": crop_amount_px,
        "crop_amount_inches": crop_amount_inches,
        "equivalent_print_size": equivalent_print_size
    }


# ============== OUTPUT FORMAT SETTINGS ==============
OUTPUT_FORMATS = ["png", "tiff"]

# ============== PROMPT SETTINGS ==============
positive_prompt = ""
positive_prompt2 = ""
negative_prompt = ""
negative_prompt2 = ""
guidance = 3.5

# ============== UPSCALE SETTINGS ==============
upscale_by = 4
seed = 0
steps = 20
cfg = 7
sampler_name = "euler"
scheduler = "normal"
denoise = 0.2

upscale_model = "4x-UltraSharp.pth"

# ============== TILE SETTINGS ==============
mode_type = "Linear"
tile_width = 512
tile_height = 512
mask_blur = 8
tile_padding = 32

# ============== SEAM FIX SETTINGS ==============
seam_fix_mode = "None"
seam_fix_denoise = 1.0
seam_fix_width = 64
seam_fix_mask_blur = 8
seam_fix_padding = 16
force_uniform_tiles = True
tiled_decode = False

# ============== LoRA SETTINGS ==============
use_loRA = False
LoRA_Strength = 1.0

# ============== FRAME REMOVAL SETTINGS ==============
remove_frame_before_upscale = False
frame_removal_threshold = 0.15
min_frame_width = 10
max_frame_width = 200

print("=" * 60)
print("2x EVALUATION MODE - Configuration")
print("=" * 60)
print(f"Model: {upscale_model}")
print(f"Upscale by: {upscale_by}x")
print(f"DPI: {DPI}")
print(f"Evaluation multiplier: {EVALUATION_MULTIPLIER}x")
print(f"Output formats: {OUTPUT_FORMATS}")
print(f"\nImages to process (2x evaluation sizes):")
for config in IMAGE_CONFIGS:
    print(f"  {config['image_id'][:8]}...")
    print(f"    Original target: {config['original_width']}\" x {config['original_height']}\"")
    print(f"    Evaluation size: {config['eval_width_inches']}\" x {config['eval_height_inches']}\" @ {DPI}DPI")
    print(f"    Output pixels: {config['target_width_px']}x{config['target_height_px']}px")

In [ ]:
# @title Run 2x Evaluation Upscaling with FLUX

def resize_to_dimensions(image_tensor: torch.Tensor, target_width: int, target_height: int) -> torch.Tensor:
    """Resize image tensor to exact target dimensions using high-quality Lanczos."""
    img_np = (image_tensor[0].cpu().numpy() * 255).astype(np.uint8)
    pil_img = Image.fromarray(img_np)
    pil_img = pil_img.resize((target_width, target_height), Image.LANCZOS)
    resized_np = np.array(pil_img).astype(np.float32) / 255.0
    resized_tensor = torch.from_numpy(resized_np).unsqueeze(0)
    return resized_tensor


def upscale_single_image(
    image_path: str,
    target_width_inches: float,
    target_height_inches: float,
    original_width_inches: float,
    original_height_inches: float,
    dpi: int,
    output_name: str,
    output_formats: list,
    positive_prompt: str,
    positive_prompt2: str,
    negative_prompt: str,
    negative_prompt2: str,
    guidance: float,
    upscale_by: float,
    seed: int,
    steps: int,
    cfg: float,
    sampler_name: str,
    scheduler: str,
    denoise: float,
    upscale_model_name: str,
    mode_type: str,
    tile_width: int,
    tile_height: int,
    mask_blur: int,
    tile_padding: int,
    seam_fix_mode: str,
    seam_fix_denoise: float,
    seam_fix_width: int,
    seam_fix_mask_blur: int,
    seam_fix_padding: int,
    force_uniform_tiles: bool,
    tiled_decode: bool,
    use_lora: bool,
    lora_strength: float,
    clip, model, vae, upscale_model_load, positive, negative
) -> dict:
    """Upscale a single image at 2x evaluation size."""
    import time
    start_time = time.time()
    
    image_path = Path(image_path)
    if not image_path.is_absolute():
        image_path = REPO_ROOT / image_path
    
    if not image_path.exists():
        raise FileNotFoundError(f"Input file not found: {image_path}")
    
    print(f"Processing: {image_path.name}")
    print(f"Original target: {original_width_inches}\" x {original_height_inches}\"")
    print(f"Evaluation size: {target_width_inches}\" x {target_height_inches}\" @ {dpi} DPI")
    
    loaded_image = load_image.load_image(str(image_path))[0]
    
    input_height, input_width = loaded_image.shape[1], loaded_image.shape[2]
    input_aspect = input_width / input_height
    target_aspect = target_width_inches / target_height_inches
    print(f"Input size: {input_width}x{input_height} (aspect: {input_aspect:.3f})")
    
    scale_info = calculate_scale_for_crop(
        input_width=input_width,
        input_height=input_height,
        target_width_inches=target_width_inches,
        target_height_inches=target_height_inches,
        dpi=dpi
    )
    
    output_width = scale_info["output_width_px"]
    output_height = scale_info["output_height_px"]
    
    print(f"\n📐 2x Evaluation Scaling:")
    print(f"   Output: {output_width}x{output_height}px ({scale_info['equivalent_print_size']})")
    
    upscaled_w = int(input_width * upscale_by)
    upscaled_h = int(input_height * upscale_by)
    print(f"\n   Step 1: FLUX upscale {upscale_by}x -> {upscaled_w}x{upscaled_h}")
    print(f"   Step 2: Resize to evaluation size -> {output_width}x{output_height}")
    
    print("\nUpscaling with FLUX...")
    upscale_start = time.time()
    
    image_out = upscaler.upscale(
        image=loaded_image,
        model=model,
        positive=positive,
        negative=negative,
        vae=vae,
        upscale_by=upscale_by,
        seed=seed,
        steps=steps,
        cfg=cfg,
        sampler_name=sampler_name,
        scheduler=scheduler,
        denoise=denoise,
        upscale_model=upscale_model_load,
        mode_type=mode_type,
        tile_width=tile_width,
        tile_height=tile_height,
        mask_blur=mask_blur,
        tile_padding=tile_padding,
        seam_fix_mode=seam_fix_mode,
        seam_fix_denoise=seam_fix_denoise,
        seam_fix_mask_blur=seam_fix_mask_blur,
        seam_fix_width=seam_fix_width,
        seam_fix_padding=seam_fix_padding,
        force_uniform_tiles=force_uniform_tiles,
        tiled_decode=tiled_decode,
    )[0]
    
    print(f"FLUX upscaling took: {time.time() - upscale_start:.1f}s")
    
    print(f"Resizing to evaluation size: {output_width}x{output_height}...")
    image_resized = resize_to_dimensions(image_out, output_width, output_height)
    
    print(f"\nSaving outputs ({', '.join(output_formats)})...")
    output_paths = save_as_image(image_resized[0], output_name, formats=output_formats)
    
    print(f"\n✅ Done!")
    print(f"Output size: {output_width}x{output_height}px")
    print(f"Evaluation print size: {scale_info['equivalent_print_size']} @ {dpi} DPI")
    print(f"Total time: {time.time() - start_time:.1f}s")
    
    return {
        "output_paths": output_paths,
        "output_width": output_width,
        "output_height": output_height,
        "equivalent_print_size": scale_info["equivalent_print_size"]
    }


# ============== BATCH PROCESSING ==============
print("=" * 60)
print("2x EVALUATION MODE - FLUX UPSCALING")
print("=" * 60)

with torch.inference_mode():
    print("\nLoading models...")
    
    print("Loading Text_Encoder...")
    clip = clip_loader.load_clip(flux_t5xxl, flux_clip_l, "flux")[0]
    
    positive = positive_prompt_encode.encode(clip, positive_prompt, positive_prompt2, guidance)[0]
    negative = negative_prompt_encode.encode(clip, negative_prompt, negative_prompt2, guidance)[0]
    
    del clip
    torch.cuda.empty_cache()
    gc.collect()
    
    print("Loading Unet Model...")
    model = unet_loader.load_unet(flux_model)[0]
    
    print("Loading upscale model...")
    upscale_model_load = upscale_model_loader.load_model(upscale_model)[0]
    
    print("Loading VAE...")
    vae = vae_loader.load_vae(flux_vae)[0]
    
    if use_loRA and lora is not None:
        print("Loading LoRA...")
        model = load_lora.load_lora_model_only(model, lora, LoRA_Strength)[0]
    
    print("\n✅ Models loaded!\n")
    
    results = []
    
    for i, config in enumerate(IMAGE_CONFIGS, 1):
        print(f"\n{'=' * 60}")
        print(f"[{i}/{len(IMAGE_CONFIGS)}] {config['image_id']}")
        print(f"Original: {config['original_width']}\" x {config['original_height']}\"")
        print(f"Evaluating at: {config['eval_width_inches']}\" x {config['eval_height_inches']}\"")
        print(f"{'=' * 60}")
        
        actual_seed = seed if seed != 0 else random.randint(0, 2**32 - 1)
        print(f"Using seed: {actual_seed}")
        
        try:
            # Use _2x-eval suffix for evaluation outputs
            output_name = f"{config['image_id']}_flux-upscaler_2x-eval"
            
            result = upscale_single_image(
                image_path=config["input_path"],
                target_width_inches=config["eval_width_inches"],
                target_height_inches=config["eval_height_inches"],
                original_width_inches=config["original_width"],
                original_height_inches=config["original_height"],
                dpi=DPI,
                output_name=output_name,
                output_formats=OUTPUT_FORMATS,
                positive_prompt=positive_prompt,
                positive_prompt2=positive_prompt2,
                negative_prompt=negative_prompt,
                negative_prompt2=negative_prompt2,
                guidance=guidance,
                upscale_by=upscale_by,
                seed=actual_seed,
                steps=steps,
                cfg=cfg,
                sampler_name=sampler_name,
                scheduler=scheduler,
                denoise=denoise,
                upscale_model_name=upscale_model,
                mode_type=mode_type,
                tile_width=tile_width,
                tile_height=tile_height,
                mask_blur=mask_blur,
                tile_padding=tile_padding,
                seam_fix_mode=seam_fix_mode,
                seam_fix_denoise=seam_fix_denoise,
                seam_fix_width=seam_fix_width,
                seam_fix_mask_blur=seam_fix_mask_blur,
                seam_fix_padding=seam_fix_padding,
                force_uniform_tiles=force_uniform_tiles,
                tiled_decode=tiled_decode,
                use_lora=use_loRA,
                lora_strength=LoRA_Strength,
                clip=None,
                model=model,
                vae=vae,
                upscale_model_load=upscale_model_load,
                positive=positive,
                negative=negative
            )
            
            results.append({
                "image_id": config["image_id"],
                "status": "Success",
                "outputs": result["output_paths"],
                "output_size": f"{result['output_width']}x{result['output_height']}",
                "eval_size": result["equivalent_print_size"],
                "original_target": f"{config['original_width']}\" x {config['original_height']}\""
            })
            
        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            results.append({
                "image_id": config["image_id"],
                "status": "Failed",
                "error": str(e)
            })
    
    del model, vae, upscale_model_load, positive, negative
    clear_memory()

# Summary
print(f"\n\n{'=' * 60}")
print("2x EVALUATION - COMPLETE")
print(f"{'=' * 60}")

success_count = sum(1 for r in results if r["status"] == "Success")
print(f"\nProcessed: {success_count}/{len(results)} images successfully\n")

for r in results:
    if r["status"] == "Success":
        print(f"✅ {r['image_id'][:8]}...")
        for out_path in r["outputs"]:
            print(f"   Output: {Path(out_path).name}")
        print(f"   Size: {r['output_size']} ({r['eval_size']} @ {DPI} DPI)")
        print(f"   Original target: {r['original_target']}")
        print()
    else:
        print(f"❌ {r['image_id'][:8]}... -> {r.get('error', 'Unknown error')}")

print("\n📋 Next steps:")
print("   1. Open the output in Photoshop")
print("   2. View at 100% to check for pixelation")
print("   3. If quality is acceptable, run the standard notebook for final output")